# XGBoost Algorithm From Scratch

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from pmlb import fetch_data
import time
import xgboost as xgb

## The XGBoost Classifier

In [3]:
class TreeNode:
    def __init__(self, depth):
        self.depth = depth
        self.left = None
        self.right = None
        self.split_feature = None
        self.split_value = None
        self.leaf_value = None

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_true, y_pred_prob):
    epsilon = 1e-15
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred_prob)) / len(y_true)

def find_best_split(X, gradients):
    best_feature, best_value, best_reduction = None, None, -float('inf')

    for feature in range(X.shape[1]):
        unique_values = np.unique(X[:, feature])
        for value in unique_values:
            left_indices = X[:, feature] <= value
            right_indices = ~left_indices

            left_gradients = gradients[left_indices]
            right_gradients = gradients[right_indices]

            reduction = np.sum(left_gradients**2) + np.sum(right_gradients**2) - np.sum(gradients**2)
            if reduction > best_reduction:
                best_feature, best_value, best_reduction = feature, value, reduction

    return best_feature, best_value

def build_tree(X, gradients, max_depth, current_depth=0):
    if current_depth == max_depth or np.sum(gradients**2) < 1e-3:
        leaf = TreeNode(depth=current_depth)
        leaf.leaf_value = np.mean(gradients, axis=0)
        return leaf

    best_feature, best_value = find_best_split(X, gradients)

    if best_feature is None:
        leaf = TreeNode(depth=current_depth)
        leaf.leaf_value = np.mean(gradients, axis=0)
        return leaf

    node = TreeNode(depth=current_depth)
    node.split_feature = best_feature
    node.split_value = best_value

    left_indices = X[:, best_feature] <= best_value
    right_indices = ~left_indices

    node.left = build_tree(X[left_indices], gradients[left_indices], max_depth, current_depth + 1)
    node.right = build_tree(X[right_indices], gradients[right_indices], max_depth, current_depth + 1)

    return node

def predict_tree(node, X):
    if node.leaf_value is not None:
        return node.leaf_value

    if X[node.split_feature] <= node.split_value:
        return predict_tree(node.left, X)
    else:
        return predict_tree(node.right, X)

class MyXGBoostClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.num_classes = None  

    def fit(self, X, y):
        X = np.array(X)

        class_labels = np.unique(y)
        num_classes = len(class_labels)
        one_hot_y = np.eye(num_classes)[y]

        predictions = np.zeros_like(one_hot_y) + np.mean(one_hot_y, axis=0) #0.5 nai ghetla
        self.num_classes = len(np.unique(y))

        for _ in range(self.n_estimators):
            gradients = one_hot_y - softmax(predictions) #log of odds chya jaagi softmax
            tree = build_tree(X, gradients, max_depth=self.max_depth)
            self.trees.append(tree)

            tree_predictions = np.array([predict_tree(tree, x) for x in X])
            predictions += self.learning_rate * tree_predictions
            #print(np.exp(predictions)/(1+np.exp(predictions)))

    def predict_proba(self, X):
        X = np.array(X)

        num_classes = self.num_classes

        predictions = np.zeros((len(X), len(self.trees), num_classes))

        for i, tree in enumerate(self.trees):
            tree_predictions = np.array([predict_tree(tree, x) for x in X])
            predictions[:, i, :] = self.learning_rate * tree_predictions

        return softmax(np.sum(predictions, axis=1))


    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)


## Evaluating Our Model

### Iris Dataset

In [3]:
iris = pd.read_csv("iris.csv")
iris = iris.drop('Id', axis=1)

X = iris.iloc[:, 0:4]
y = iris['Species']
labels = {item: index for index, item in enumerate(np.unique(y))}
y = y.map(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, shuffle=True)

model = MyXGBoostClassifier(n_estimators=100, learning_rate=0.5, max_depth=3)
start_time = time.perf_counter()
model.fit(X_train, y_train)
end_time = time.perf_counter()
predictions = model.predict(X_test)
elapsed_time_microsec = (end_time - start_time) * 1000

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print(f"Training Time: {elapsed_time_microsec:.2f} milliseconds")

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Accuracy: 0.9666666666666667
Training Time: 3019.93 milliseconds


### Breast Cancer Dataset

In [4]:
breast_cancer = pd.read_csv("data.csv")
breast_cancer = breast_cancer.drop('id', axis=1)

X = breast_cancer.iloc[:, 2:]
y = breast_cancer['diagnosis']
labels = {item: index for index, item in enumerate(np.unique(y))}
y = y.map(labels)



X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, shuffle=True)

from sklearn.impute import SimpleImputer


imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train_imputed = imputer.fit_transform(X_train)



model = MyXGBoostClassifier(n_estimators=5, learning_rate=0.5, max_depth=10)
start_time = time.perf_counter()
model.fit(X_train_imputed, y_train)
end_time = time.perf_counter()

X_test_imputed = imputer.transform(X_test)
predictions = model.predict(X_test_imputed)
elapsed_time_microsec = (end_time - start_time) * 1000


accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print(f"Training Time: {elapsed_time_microsec:.2f} milliseconds")

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Accuracy: 0.8333333333333334
Training Time: 49606.25 milliseconds


### Ann Thyroid Dataset

In [5]:
X3, y3 = fetch_data('ann_thyroid', return_X_y=True, local_cache_dir='./')

conditions = [y3 == 1, y3 == 2, y3 == 3]
values = [0, 1, 2]

y3 = np.select(conditions, values, default=y3)

X_train, X_test, y_train, y_test = train_test_split(X3, y3, random_state=10, test_size=0.2, shuffle=True)

model = MyXGBoostClassifier(n_estimators=10, learning_rate=0.1, max_depth=3)
start_time = time.perf_counter()
model.fit(X_train, y_train)
end_time = time.perf_counter()

elapsed_time_microsec = (end_time - start_time) * 1000

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print(f"Training Time: {elapsed_time_microsec:.2f} milliseconds")

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Accuracy: 0.9215277777777777
Training Time: 18964.69 milliseconds


## Using XGBoost from library

### Iris Dataset

In [7]:
iris = pd.read_csv("iris.csv")
iris = iris.drop('Id', axis=1)

X = iris.iloc[:, 0:4]
y = iris['Species']
labels = {item: index for index, item in enumerate(np.unique(y))}
y = y.map(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, shuffle=True)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "multi:softmax", 
    "num_class": len(np.unique(y)),
    "max_depth": 3,                
    "learning_rate": 0.5,                     
}

start_time = time.perf_counter()
model = xgb.train(params, dtrain)

end_time = time.perf_counter()

elapsed_time_microsec = (end_time - start_time) * 1000

y_pred = model.predict(dtest)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(f"Training Time: {elapsed_time_microsec:.2f} milliseconds")

Accuracy: 1.0
Training Time: 19.49 milliseconds


### Breast Cancer Dataset

In [6]:
X2, y2 = fetch_data('breast_cancer', return_X_y=True, local_cache_dir='./')

X_train, X_test, y_train, y_test = train_test_split(X2, y2, random_state=10, test_size=0.2, shuffle=True)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "multi:softmax",  
    "num_class": len(np.unique(y2)),
    "max_depth": 3,                 
    "learning_rate": 0.1,                    
}

start_time = time.perf_counter()
model = xgb.train(params, dtrain)
end_time = time.perf_counter()

elapsed_time_microsec = (end_time - start_time) * 1000

y_pred = model.predict(dtest)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Training Time (ms):", elapsed_time_microsec)


Accuracy: 0.7758620689655172
Training Time (ms): 9.48540001991205


### Ann Thyroid Dataset

In [5]:
X4, y4 = fetch_data('ann_thyroid', return_X_y=True, local_cache_dir='./')

conditions = [y4 == 1, y4 == 2, y4 == 3]
values = [0, 1, 2]

y4 = np.select(conditions, values, default=y4)

X_train, X_test, y_train, y_test = train_test_split(X4, y4, random_state=10, test_size=0.2, shuffle=True)


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


params = {
    "objective": "multi:softmax",  
    "num_class": len(np.unique(y4)),       
    "max_depth": 3,                  
    "learning_rate": 0.1,                         
}


start_time = time.perf_counter()
model = xgb.train(params, dtrain)
end_time = time.perf_counter()

elapsed_time_millisec = (end_time - start_time) * 1000

y_pred = model.predict(dtest)
y_pred_int = y_pred.astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred_int))
print(f"Training Time: {elapsed_time_millisec:.2f} milliseconds")

Accuracy: 0.9909722222222223
Training Time: 24.34 milliseconds
